## Canciones en Spotify entre 1958 y 2019

Spotify, en su conjunto, tiene una biblioteca de 30 millones de canciones. No solo contiene estas canciones, sino que también define y documenta los atributos de cada una de estas canciones (el tipo de atributos se aclarará a continuación). Esta es una cantidad increíble de información que requiere una arquitectura de Big Data para ser utilizada correctamente. Y es fundamental contar con esta arquitectura porque existe un inmenso potencial para la monetización de estos datos dentro de la industria de la música.

El conjunto de datos final que se debe utilizar en el análisis contiene 277.965 filas y 30 columnas para 24.000 canciones. La clave primaria de los datos es una combinación de WeekID y SongID.

* url: la URL de la canción en el sitio de la cartelera.
* WeekID: la semana para la que se registra el rendimiento de la canción
* Week Position: la posición que ocupó la canción durante esa semana (varía de 1 a 100)
* Song: título de la canción
* Performer: nombre del intérprete
* SongID: identificador de Billboard para la canción (concatenación de columnas anteriores)
* Instance: número de veces que se ha producido
* Previous Week Position: posición de la semana pasada
* Peak Position: la mejor posición que jamás haya ocupado
* Weeks on Chart: cuántas semanas ha estado en el top 100
* top10_tag: si la posición ocupada estaba dentro del top 10
* spotify_genre: género personalizado de Spotify (lista por canción)
* spotify_track_id: identificación alfanumérica
* spotify_track_preview_url: versión preliminar breve de la URL de la pista
* spotify_track_album – Álbum al que pertenece la canción
* spotify_track_explicit: registro de etiquetas si la pista contiene lenguaje explícito o no
* spotify_track_duration_ms: duración de la canción en milisegundos
* spotify_track_popularity: qué tan popular es la pista en Spotify
* danceability: métrica que define la probabilidad de que la canción se pueda bailar
* energy: Métrica que define qué tan enérgica es la canción
* key: tonalidad musical en la que se reproduce la canción
* loudness: métrica que define el volumen de la canción
* mode: pregrabación estéreo o mono
* speechiness: Métrica que define cuánto canto vs instrumento hay en la canción
* acousticness: métrica que define el grado en que se utilizan los instrumentos acústicos.
* instrumentalness: métrica que define cuánto de la canción utiliza instrumentos
* liveness: métrica que define el estado de ánimo de una canción
* valence: Métrica que define la positividad de la canción
* tempo: Métrica que define el tempo de la canción
* time_signature: métrica para etiquetar el tiempo de la canción

**Nombre completo del alumno:**

**INSTRUCCIONES**: en cada celda debes responder a la pregunta formulada, asegurándote de que el resultado queda guardado en la(s) variable(s) que por defecto vienen inicializadas a `None`. No se necesita usar variables intermedias, pero puedes hacerlo siempre que el resultado final del cálculo quede guardado exactamente en la variable que venía inicializada a None (debes reemplazar None por la secuencia de transformaciones necesarias, pero nunca cambiar el nombre de esa variable). **No olvides borrar la línea *raise NotImplementedError()* de cada celda cuando hayas completado la solución de esa celda y quieras probarla**.

Después de cada celda evaluable verás una celda con código. Ejecútala (no modifiques su código) y te dirá si tu solución es correcta o no. Además de esas pruebas, se realizarán algunas más (ocultas) a la hora de puntuar el ejercicio, pero evaluar dicha celda es un indicador bastante fiable acerca de si realmente has implementado la solución correcta o no. Asegúrate de que, al menos, todas las celdas indican que el código es correcto antes de enviar el notebook terminado.

*No olvides todas las sentencias import necesarias. Una celda que no pueda ser ejecutada por faltar import no será evaluada*

### Sobre el dataset anterior (spotify_dataset.csv) se pide:

**Ejercicio 1 (1 punto)** Leerlo tratando de que Spark infiera el tipo de dato de cada columna, sin cachearlo.
* Puesto que existen columnas que contienen una coma enmedio del valor, en esos casos los valores vienen entre comillas dobles. Spark ya contempla esta posibilidad y puede leerlas adecuadamente **si al leer le indicamos las siguientes opciones adicionales** además de las que ya sueles usar: `.option("quote", "\"").option("escape", "\"")`.
* Asegúrate de que las **filas que no tienen el formato correcto sean descartadas**, indicando también la opción `mode` con el valor `DROPMALFORMED` como vimos en clase.

In [1]:
# LÍNEA EVALUABLE, NO RENOMBRAR LAS VARIABLES
import pyspark.sql.functions as F

spotify_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .option("mode", "DROPMALFORMED") \
    .csv("gs://core-memento-378812/notebooks/jupyter/spotify_dataset.csv")


In [2]:
assert(spotify_df.count() == 277965)
assert(spotify_df.is_cached == False)

**Ejercicio 2 (2 puntos)** Limpiar y enriquecer las variables aplicando las siguientes transformaciones encadenadas a `spotify_df`:
* Reemplazar la columna `WeekID` por su versión convertida a tipo fecha (DateType). Esto se hará en una sola línea:
  * Primero se convierte en una columna de Timestamp  con `F.from_unixtime(F.unix_timestamp('nombreColumna', 'dd-MM-yyyy')`
  * En esa misma línea, dicha columna debe ser convertida a tipo DateType usando el método de la clase Column visto en clase.
* Crear una nueva columna `title_length` con la longitud (número de caracteres) del título de cada canción. PISTA: consulta la [documentación oficial](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.length.html) de la función `F.length`.
* Reemplazar la columna `Performer` por el resultado de *cortar cada título por el string* `" & "` (el carácter & precedido y seguido de un espacio en blanco), lo cual te devolverá una columna de vectores de string. PISTA: consulta la [documentación oficial](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.split.html) de la función `F.split`. En el caso de no haber ningún & el vector devuelto tendrá 1 elemento.
* Crear una nueva columna `n_performers` con el número de intérpretes de una canción. Para ello, debes *contar el número de elementos* de cada vector de la columna `Performer` y [la documentación](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.size.html) de la función `F.size` (ojo, no confundir con `length` utilizada antes). Ambas se pueden concatenar ya que actúan sobre un objeto Column y devuelven otro objeto Column. No te preocupes por los elementos nulos que hubiese en esta columna.
* Añadir dos columnas `year` y `month` que contengan respectivamente el año y el mes al que corresponde la fecha de la columna `WeekID`. PISTA: consulta la documentación oficial de [la función F.year](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.year.html) y de [la función F.month](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.month.html). 
* Guardar el resultado de las tres transformaciones anteriores en una nueva variable `spotify_enriquecido_df` y **cachearla**.

* Guardar en la variable `fila_mas_autores` el objeto Row de la canción con el **mayor número de intérpretes**, deshaciendo los posibles empates por la que tenga la **menor longitud en el título** (cuidado: los dos criterios de ordenación son independientes y contrapuestos. Recuerda las funciones `asc()` y `desc()` de los objetos columna). PISTA: utilizar la acción `first()` tras ordenar el dataframe `spotify_enriquecido_df`, el cual no está ni debe quedar ordenado, puesto que la versión ordenada de dicho dataframe no debe reemplazar a dicha variable, sino que puedes usar otra variable auxiliar para guardarla y aplicar `first()` sobre ella.
* Accediendo a los campos del objeto anterior, almacenar en la variable `titulo_mas_autores` y `url_mas_autores` los campos del título de la canción y su URL, respectivamente.

In [3]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, asc,desc

spotify_enriquecido_df = spotify_df.withColumn("WeekID", F.to_date(F.from_unixtime(F.unix_timestamp('WeekID', 'dd-MM-yyyy'), 'yyyy-MM-dd'))) \
                                   .withColumn("title_length", F.length(col("song"))) \
                                   .withColumn("Performer", F.split(col("Performer"), ' & ')) \
                                   .withColumn("n_performers", F.size(col("Performer"))) \
                                   .withColumn("year", F.year("WeekID")).withColumn("month", F.month("WeekID"))


spotify_enriquecido_ordenado_df = spotify_enriquecido_df.orderBy(desc("n_performers"), asc("title_length"))
spotify_enriquecido_df.cache()

fila_mas_autores = spotify_enriquecido_ordenado_df.first()

titulo_mas_autores = fila_mas_autores["Song"]
url_mas_autores = fila_mas_autores["url"]


23/05/11 16:45:33 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
tipos = dict(spotify_enriquecido_df.dtypes)
assert(tipos["title_length"] == "int" and 
       tipos["Performer"] == "array<string>" and 
       tipos["n_performers"] == "int")
assert(sum([1 for x in ["title_length", "n_performers", "year", "month"] if x in spotify_enriquecido_df.columns]) == 4)
assert(fila_mas_autores.n_performers == 3)
assert(fila_mas_autores.url == "http://www.billboard.com/charts/hot-100/1998-08-01")
r = spotify_enriquecido_df.select(F.mean("n_performers").alias("n_performers"),
                                  F.mean("title_length").alias("title_length")).first()
assert(round(r.n_performers, 2) == 1.07)
assert(round(r.title_length, 2) == 15.59)


**Ejercicio 3 (1.5 puntos)** Partiendo del DataFrame `spotify_enriquecido_df` se pide:

* Crear un nuevo DataFrame que tenga tantas filas como años distintos aparecen, y tantas columnas como meses del año más una (que es el año y debe estar a la izquierda del todo). Las columnas deben aparecer de izqda a dcha del 1 a 12 -recuerda que el método `select()` te las devuelve en el orden que se las pidas-. En cada casilla de dicho DF debe aparecer el *número de canciones* ***distintas*** que han estado durante ese mes en algún momento en el Top 10. Se deben contar canciones distintas atendiendo a la columna `SongID`.
  * PISTA: el punto de partida debe ser el DF formado solamente por aquellas semanas que están en el top10. A partir de ahí debes construir la tabla anterior. Recuerda qué función de `pyspark.sql.functions` te sirve para contar el número de elementos **distintos** de una columna o en este caso, de un grupo.
  * PISTA: examina los valores posibles de la columna `top10_tag` para deducir con cuáles debes quedarte como punto de partida.
* Rellenar los valores nulos del DataFrame resultante con 0.
* Tras rellenarlos, añadir una nueva columna `n_top10_promedio` que contenga para cada año (fila) el número medio (redondeado a 2 cifras decimales con la función `F.round`) de canciones distintas que hay en todos los meses de ese año. 
  * PISTA: esta operación se realiza exclusivamente mediante operaciones aritméticas con columnas. No se necesita F.mean ni F.sum ni nada parecido. La función `F.round` se debe aplicar al objeto columna resultante de la operación aritmética.
* Las filas del DataFrame resultante deben quedar ordenadas de menor a mayor año.
* Guardar el resultado de las transformaciones anteriores en una nueva variable `distintos_top10_df.

In [5]:
from pyspark.sql.functions import countDistinct, round, count, year, month

distintos_top10_df = (
    spotify_enriquecido_df 
    .filter(col('top10_tag') == 'Top 10')
    .groupBy(year('WeekID').alias('year'), month('WeekID').alias('month'))
    .agg(countDistinct('SongID').alias('num_canciones'))
    .cache()
)

years = distintos_top10_df.select('year').distinct().orderBy('year')
months_cols = [str(i) for i in range(1, 13)]
columns = ['year'] + months_cols

distintos_top10_df =distintos_top10_df.groupBy('year').pivot('month', months_cols).sum('num_canciones') \
    .orderBy('year')


distintos_top10_df = years.join(distintos_top10_df, on='year', how='left').fillna(0)

distintos_top10_df = distintos_top10_df.withColumn(
    'n_top10_promedio',
    round((col('1') + col('2') + col('3') + col('4') + col('5') + col('6') +
           col('7') + col('8') + col('9') + col('10') + col('11') + col('12')) / 12, 2)
)

distintos_top10_df =distintos_top10_df.orderBy(col('year'))



In [6]:
lista = distintos_top10_df.take(5)
assert(lista[0].year == 1958 and lista[0]["12"] == 12 and lista[0].n_top10_promedio == 9.42)
assert(lista[2].year == 1960 and lista[2]["4"] == 21 and lista[2].n_top10_promedio == 24.58)

**Ejercicio 4 (1.5 puntos)** Partiendo de nuevo del DataFrame almacenado en `spotify_enriquecido_df` se pide:

* Reemplazar la columna `top10_tag`, que actualmente son "Top 10" y "Other" (literalmente) por una versión recategorizada (usando `F.when`) cuyos valores sean los *números enteros* 1 y 0, respectivamente (es decir, Top 10 -> 1 y Other -> 0). No es necesario usar F.lit, aunque puedes hacerlo.
* Crear una nueva columna `semanas_top10` que contenga para cada canción el número **total** de semanas que ha estado en el Top 10 **en ese mismo año** al que corresponde la fila. Está prohibido utilizar JOIN. Debe resolverse obligatoriamente creando una ventana en la  variable `w` (ATENCIÓN: hay que contar para **cada canción** y **ese mismo año**).
  * PISTA: vas a necesitar la columna `top10_tag` que has recategorizado como enteros.
  * PISTA: recuerda importar la función Window de `pyspark.sql`.
* A continuación, seleccionar las columnas `SongID`, `Performer` y `semanas_top10` y quitar duplicados.
* Ordenar el DataFrame resultante descentemente en base a `semanas_top10` y desempatar descendentemente en base a `SongID` (Spark utilizará orden alfabético al ser de tipo string).
* Guardar el resultado de las transformaciones anteriores en la variable `semanas_top10_df`.

In [7]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df_recategorizado = spotify_enriquecido_df.withColumn("top10_tag_recategorizado",
                                                     F.when(F.col("top10_tag") == "Top 10", 1)
                                                     .otherwise(0))

w = Window.partitionBy("SongID", F.year("WeekID"))

df_con_semanas_top10 = df_recategorizado.withColumn("semanas_top10",
                                                    F.sum("top10_tag_recategorizado").over(w))

df_resultado = df_con_semanas_top10.select("SongID", "Performer", "semanas_top10").distinct().dropDuplicates()

df_resultado_ordenado = df_resultado.orderBy(F.desc("semanas_top10"), F.desc("SongID"))
semanas_top10_df = df_resultado_ordenado



In [8]:
lista = semanas_top10_df.take(4)
assert(lista[0].semanas_top10 == 30 and "Bad Guy" in lista[0].SongID)
assert(lista[1].semanas_top10 == 28)
assert(lista[2].semanas_top10 == 28)
assert(lista[3].semanas_top10 == 27 and "Girls Like You" in lista[3].SongID)

**Ejercicio 5 (1 punto)** Partiendo del DataFrame almacenado en `semanas_top10_df`:

* Reemplazar la columna `Performer` por el resultado de *explotarla* utilizando la [función](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.explode.html) `F.explode(...)`, esto es,  desenrollar los vectores que hay en dicha columna, de manera que cada valor del vector pase a tener su propia fila completa. Por ejemplo si una casilla de esa columna contiene el vector \["Beyonce", "Pitbull"\] porque hayan colaborado juntos en alguna canción, la operación `F.explode` sobre esa columna dará lugar a dos filas en el DataFrame resultante, que serán idénticas entre sí en todas las columnas excepto en la columna `Performer`, donde una de las dos filas tendrá el valor "Beyonce" y la otra fila tendrá el valor "Pitbull". El DataFrame resultante tendrá probablemente bastantes más filas que el original.
* A continuación, encadenando una transformación aplicada al DF devuelto por el apartado anterior, crear un nuevo DataFrame con tantas filas como intérpretes distintos y dos columnas, que sean el nombre dintérprete y el número **total** de semanas que sus canciones han estado en el top 10, llamando a dicha columna `total_semanas_top10` (poniendo sobre la marcha en el momento de crearla).
* Ordenar dicho DataFrame descendentemente en base a `total_semanas_top10`, y guardar el resultado en una variable `top_performers_df`
* Utilizar una acción que lleve al Driver, a la variable `top_performers`, una lista de Python con el top 3 de estos artistas. ¿Te suenan...?

In [9]:
from pyspark.sql.functions import explode

exploded_df = semanas_top10_df.withColumn("Performer", F.explode(semanas_top10_df.Performer))
top_performers_df = exploded_df.groupBy("Performer").agg(F.count("*").alias("total_semanas_top10")).orderBy("total_semanas_top10", ascending=False)


top_performers = [row[0] for row in top_performers_df.select("Performer").take(3)]

top_performers_df.show()
exploded_df.show()

+------------------+-------------------+
|         Performer|total_semanas_top10|
+------------------+-------------------+
|         Glee Cast|                178|
|             Drake|                 97|
|      Taylor Swift|                 91|
|        Elton John|                 66|
|       The Beatles|                 65|
|           Madonna|                 64|
|   Aretha Franklin|                 60|
|     Stevie Wonder|                 59|
|The Rolling Stones|                 57|
|       Marvin Gaye|                 55|
|        Diana Ross|                 55|
|    The Beach Boys|                 52|
|              Cher|                 51|
|         Lil Wayne|                 50|
|           Chicago|                 50|
|   Michael Jackson|                 50|
|      Neil Diamond|                 49|
|            Future|                 49|
|   The Temptations|                 49|
|    Dionne Warwick|                 48|
+------------------+-------------------+
only showing top

In [10]:
assert(top_performers_df.count() == 7992)
assert(top_performers[0].Performer == "Mariah Carey" and top_performers[0].total_semanas_top10 == 208)
assert(top_performers[1].Performer == "Madonna" and top_performers[1].total_semanas_top10 == 205)
assert(top_performers[2].Performer == "The Beatles" and top_performers[2].total_semanas_top10 == 182)

AttributeError: 'str' object has no attribute 'Performer'

Este ejercicio y los dos siguientes plantean un algoritmo de **clustering (K-means)** sobre las canciones, basándonos solamente en los atributos numéricos que describen los parámetros de cada canción. Concretamente, usaremos solamente las siguientes 10 columnas:

* danceability: métrica que define la probabilidad de que la canción se pueda bailar
* energy: Métrica que define qué tan enérgica es la canción
* loudness: métrica que define el volumen de la canción
* mode: pregrabación estéreo o mono (tiene dos valores: 0 y 1)
* speechiness: Métrica que define cuánto canto vs instrumento hay en la canción
* acousticness: métrica que define el grado en que se utilizan los instrumentos acústicos.
* instrumentalness: métrica que define cuánto de la canción utiliza instrumentos
* liveness: métrica que define el estado de ánimo de una canción
* valence: Métrica que define la positividad de la canción
* tempo: Métrica que define el tempo de la canción

Puesto que K-means con distancia euclídea requiere que todas las variables estén aproximadamente en el mismo rango para que todas influyan por igual en la distancia, primero vamos a reescalar todas las variables anteriores mediante un MinMaxScaler de Spark. Eso significa que tras el reescalado, todas tendrán valores entre 0 y 1. Como el MinMaxScaler requiere una columna de tipo vector como entrada porque es capaz de reescalar de una sola vez muchas features, construiremos **antes del reescalado** una columna de tipo vector con las features originales. La salida del reescalado sigue siendo una columna de tipo vector, tal como exige el algoritmo K-means que entrenaremos después.

**Ejercicio 6 (1 punto)** Partiendo del DataFrame `spotify_enriquecido_df`, tal como fue leído del fichero CSV, se pide:

* Ejecutar el código dado para crear el DataFrame `spotify_numeric_df`. No debes resolver nada en este momento, sino dejar este código sin modificar.
* Crear en la variable `vector_assembler` un VectorAssembler que reciba como entradas (argumento `inputCols`) la lista `cluster_vars` que ya viene creada, y como salida cree una nueva columna llamada `feats_vectorizadas`.
* Crear en la variable `minmax_scaler` un [objeto `MinMaxScaler()` de Spark](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.MinMaxScaler.html) que esté configurado para utilizar como entrada la columna `feats_vectorizadas`, y que cree una nueva columna de tipo vector llamada `feats_reescaladas`.
  * **NOTA IMPORTANTE**:  `MinMaxScaler` recibe como única columna de entrada una columna de tipo vector, ya que es capaz de re-escalar de una sola vez (y de manera independiente) un conjunto de columnas (o mejor dicho, una columna de tipo vector que en realidad representa varias columnas independientes ya colapsadas). Por eso en este caso particular, la penúltima pieza del pipeline, que va justo antes del algoritmo K-Means, **NO** va a ser el `vector_assembler` como suele ser habitual, sino el `minmax_scaler`, y por eso el `vector_assembler` debe ir antes de dicha pieza, ya que es quien va a crear la columna de vectores que después recibirá el `minmax_scaler` como entrada.
  * NOTA: los parámetros de valor mínimo y máximo que debemos indicar al MinMaxScaler como extremos de la columna que va a crear nueva, vienen ya fijados por defecto a 0 y 1 respectivamente (es decir, cada columna se va a re-escalar, dando lugar a otra nueva que va entre 0 y 1). Esto es justo lo que queremos para K-means, así que los dejamos sin especificar para que tomen esos valores.
* Crear en la variable `kmeans`un [estimador K-Means](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.clustering.KMeans.html) con argumento `k=10` clusters, que reciba como entrada la columna `feats_reescaladas` creada en la etapa del vector assembler, y genere como salida (argumento `predictionCol`) una nueva columna llamada `cluster`, que haga como máximo 200 iteraciones (argumento `maxIter`) y que tenga semilla 12345 (argumento `seed`, imprescindible para que coincidan los resultados de los tests).
  * A priori desconocemos el número de clusters que necesitamos, que no tiene por qué coincidir con el número de géneros musicales ya que también estamos juntando datos de muchos años diferentes y esto también influye. Deberíamos hacer un análisis con regla del codo y tratar de interpretar los clusters para varios valores de k distintos hasta quedar contentos con un valor de k, pero para simplificar, omitiremos este proceso.

In [11]:
from pyspark.ml.feature import MinMaxScaler, VectorAssembler   # recuerda los imports necesarios para MinMaxScaler y VectorAssembler
from pyspark.ml.clustering import KMeans           # recuerda el import necesario para el algoritmo KMeans

cluster_vars = ["danceability", "energy", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", 
                "liveness", "valence", "tempo"]

spotify_numeric_df = spotify_enriquecido_df\
    .groupBy("SongID").agg(*[F.first(c).alias(c) for c in ["year", "spotify_track_duration_ms"] + cluster_vars])\
    .distinct()\
    .cache()

minmax_scaler = MinMaxScaler(
    inputCol="feats_vectorizadas",
    outputCol="feats_reescaladas"
)

vector_assembler = VectorAssembler(
    inputCols=cluster_vars,
    outputCol="feats_vectorizadas"
)
kmeans = KMeans(
    k=10,
    featuresCol="feats_reescaladas",
    predictionCol="cluster",
    maxIter=200,
    seed=12345
)


In [12]:
assert(spotify_numeric_df.count() == 23751)
assert(len(spotify_numeric_df.columns) == 13)
assert(minmax_scaler.getInputCol() == "feats_vectorizadas")
assert(minmax_scaler.getOutputCol() == "feats_reescaladas")
assert(vector_assembler.getInputCols() == cluster_vars)
assert(kmeans.getK() == 10 and kmeans.getFeaturesCol() == "feats_reescaladas" 
       and kmeans.getPredictionCol() == "cluster" and kmeans.getMaxIter() == 200
       and kmeans.getSeed() == 12345)

**Ejercicio 7 (2 puntos)** 
* Crear un pipeline en la variable `pipeline` que tenga las tres etapas anteriores (el vector_assembler, el minmax_scaler y el kmeans) por ese orden.
* Entrenar el pipeline sobre el DF `spotify_numeric_df` y guardar el resultado (pipeline entrenado) en la variable `pipeline_model`.
* Aplicar el pipeline entrenado para transformar el DataFrame `spotify_numeric_df` y guardar el resultado en la variable `clustered_songs_df`, que debe quedarse **cacheada**.
* Analizar el resultado obtenido mostrando métricas agregadas de las variables dentro de cada cluster. Para ello, a partir de `clustered_songs_df`, crear en la variable `metricas_clusters_pd` un nuevo DataFrame con tantas filas como clusters (10) y que tenga 13 columnas que serán:
  * La columna más a la izquierda el cluster al que corresponde la fila, que es por lo que estamos agrupando.
  * La segunda será la mediana de la columna `year` en cada grupo. Dicha columna no se utilizó para el clustering pero existe en el DF `clustered_songs_df` 
  * Las siguientes 10 columnas deben ser la mediana de cada una de las 10 variables numéricas utilizadas para el cluster (excepto para "mode" que debemos usar la media y no la mediana)
  * La última columna debe ser el recuento del número de filas de cada grupo, que debe renombrarse como `n`. 
* Tras el cálculo, el DataFrame resultante debe pasarse a DataFrame de **pandas**, y eso es lo que debe guardarse en `metricas_clusters_pd`. 
  * Se debe usar la **mediana** de cada columna en cada grupo. Puesto que la mediana no está disponible en la API de columnas en Spark 2.4 (se incluyó en versiones posteriores) pero sí que está disponible en la API de SQL puro, tenemos que utilizar para cada agregación la expresión `F.expr("percentile_approx(nombreColumna, 0.5)").alias(nombreColumna)` que devuelve un objeto Column con la agregación y ya renombrado para llamarse igual que la columna a la que le estamos aplicando la mediana. 
  * En la columna `mode` se debe usar la **media** habitual, no la mediana, también renombrada para que se llame "mode" tras aplicar la agregación.
  * NOTA: estamos haciendo clustering, con lo que no hay ninguna división en train y test ni nada parecido! :-)

In [13]:
from pyspark.ml import Pipeline


pipeline = Pipeline(stages=[vector_assembler, minmax_scaler, kmeans])

pipeline_model = pipeline.fit(spotify_numeric_df)

clustered_songs_df = pipeline_model.transform(spotify_numeric_df).cache()

metricas_clusters_df = clustered_songs_df.groupBy("cluster").agg(
    F.expr("percentile_approx(year, 0.5)").alias("year"),
    *[F.expr("percentile_approx({}, 0.5)".format(col)).alias(col) for col in cluster_vars if col != "mode"],
    F.mean("mode").alias("mode"),
    F.count("*").alias("n")
)

metricas_clusters_pd = metricas_clusters_df.toPandas()


23/05/11 16:47:26 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/05/11 16:47:26 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [15]:
assert(metricas_clusters_pd.shape == (10, 13))
columnas = ["cluster", "year", "n"] + cluster_vars   # todas estas columnas tienen que existir en metricas_clusters_pd
assert(sum([1 for x in columnas if x in metricas_clusters_pd.columns]) == 13)